# Import

In [166]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
import seaborn as sns
import yaml
import importlib
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, accuracy_score
from imblearn.over_sampling import SMOTE

import utils

In [167]:
importlib.reload(utils)

<module 'utils' from '/home/alexm/sdd/Hackathon_nov/Hackathon_Heart_Rate_Los_Tigros/utils.py'>

# Préparation des données

In [168]:
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")

In [169]:
X_train,y_train, features_99, scaler = utils.data_processing_train(df_train)
print(X_train.head())
print(y_train.head())

Feature: PHYSHLTH
['1 - 30', '77', '88', '99', 'BLANK']
Feature: MENTHLTH
['1 - 30', '77', '88', '99', 'BLANK']
Feature: POORHLTH
['1 - 30', '77', '88', '99', 'BLANK']
Feature: SLEPTIM1
['1 - 24', '77', '99', 'BLANK']
Feature: WEIGHT2
['50 - 0776', '7777', '9023 - 9352', '9999', 'BLANK']
Feature: HEIGHT3
['200 - 711', '7777', '9061 - 9998', '9999', 'BLANK']
Feature: MARIJAN1
['1 - 30', '77', '88', '99', 'BLANK']
Feature: ALCDAY4
['101 - 199', '201 - 299', '777', '888', '999', 'BLANK']
Feature: AVEDRNK3
['1 - 76', '77', '88', '99', 'BLANK']
Feature: DRNK3GE5
['1 - 76', '77', '88', '99', 'BLANK']
Feature: MAXDRNKS
['0', '1 - 76', '77', '99', 'BLANK']
Feature: COPDSMOK
['1 - 76', '77', '88', '99', 'BLANK']
Feature: _PACKDAY
['0 - 100', 'BLANK']
    _STATE  GENHLTH  PHYSHLTH  MENTHLTH  POORHLTH  PRIMINSR  PERSDOC3  \
ID                                                                      
0        1        3  1.518224 -0.419364 -1.008502         3         1   
1        6        3 -0.409570

In [170]:
X_test = utils.data_processing_test(df_test,features_99, scaler)
print(X_test.head())

Feature: PHYSHLTH
['1 - 30', '77', '88', '99', 'BLANK']
Feature: MENTHLTH
['1 - 30', '77', '88', '99', 'BLANK']
Feature: POORHLTH
['1 - 30', '77', '88', '99', 'BLANK']
Feature: SLEPTIM1
['1 - 24', '77', '99', 'BLANK']
Feature: WEIGHT2
['50 - 0776', '7777', '9023 - 9352', '9999', 'BLANK']
Feature: HEIGHT3
['200 - 711', '7777', '9061 - 9998', '9999', 'BLANK']
Feature: MARIJAN1
['1 - 30', '77', '88', '99', 'BLANK']
Feature: ALCDAY4
['101 - 199', '201 - 299', '777', '888', '999', 'BLANK']
Feature: AVEDRNK3
['1 - 76', '77', '88', '99', 'BLANK']
Feature: DRNK3GE5
['1 - 76', '77', '88', '99', 'BLANK']
Feature: MAXDRNKS
['0', '1 - 76', '77', '99', 'BLANK']
Feature: COPDSMOK
['1 - 76', '77', '88', '99', 'BLANK']
Feature: _PACKDAY
['0 - 100', 'BLANK']
        _STATE  GENHLTH  PHYSHLTH  MENTHLTH  POORHLTH  PRIMINSR  PERSDOC3  \
ID                                                                          
225000       4        1 -0.409570 -0.419364  1.106724         1         1   
225001      35   

In [171]:
# df_temp = pd.concat([X_train, y_train])

# df_majority = df_temp[df_temp.TARGET == 0]
# df_minority = df_temp[df_temp.TARGET == 1]

# target_ratio = 0.35  # par exemple 35%

# # Nombre de lignes totales à avoir pour atteindre ce ratio
# n_total_target = int(len(df_majority) / (1 - target_ratio))

# # Nombre de minoritaires à avoir au total
# n_minority_target = n_total_target - len(df_majority)

# # Nombre à ajouter
# n_to_add = max(0, n_minority_target - len(df_minority))


# # Dupliquer les minoritaires
# df_minority_upsampled = df_minority.sample(n=n_to_add, replace=True, random_state=42)

# # Recombiner
# df_balanced = pd.concat([df_majority, df_minority_upsampled])
# df_balanced = df_balanced.sample(frac=1, random_state=42)  # mélanger
# print(df_balanced['TARGET'].value_counts())

In [172]:
# X_train = df_balanced.drop('TARGET', axis=1)
# y_train = df_balanced['TARGET'].astype(int)

In [173]:
X_train_train, X_train_test, y_train_train, y_train_test =  train_test_split(X_train, y_train, test_size = 0.2)

In [174]:
ratio = (y_train == 0).sum() / (y_train ==1).sum()
xgb = XGBClassifier(
    learning_rate=0.2,
    n_estimators=30,
    max_depth=4,
    min_child_weight=2,
    gamma=2,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha= 0.1,
    objective='binary:logistic',
    scale_pos_weight=ratio,
    seed=27,
    nthread=2
)




xgb.fit(X_train_train, y_train_train)

y_proba = xgb.predict_proba(X_train_test)[:, 1]

threshold = 0.71
y_pred = (y_proba > threshold).astype(int)

print(f1_score(y_train_test, y_pred))
print(accuracy_score(y_train_test, y_pred))

0.4186391905690151
0.8608222222222223


# Recherche best param

In [10]:
param_test1 = {
 'learning_rate':[0.1,0.2,0.3,],
 'n_estimators':[10,20,30],
 'max_depth':[1,2,3,4]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(), param_grid = param_test1, cv=2)

gsearch1.fit(X_train, y_train.values)

print("Meilleurs hyperparamètres :", gsearch1.best_params_)


Meilleurs hyperparamètres : {'learning_rate': 0.3, 'max_depth': 2, 'n_estimators': 30}


In [37]:
param = {
    "learning_rate" :0.3,
    "n_estimators":30,
    "max_depth":4,
    "min_child_weight":2
}

param_test2 = {
 'min_child_weight':[1,2,3,4,5]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier(**param), param_grid = param_test2, cv=2)
gsearch2.fit(X_train, y_train.values)


print("Meilleurs hyperparamètres :", gsearch2.best_params_)


Meilleurs hyperparamètres : {'min_child_weight': 2}


In [ ]:
param = {
    "learning_rate" :0.3,
    "n_estimators":30,
    "max_depth":4,
    "gamma":2,
    "subsample":0.9,
    "min_child_weight":2
}

param_test3 = {
    "gamma":[1,2,3],
    "subsample":[0.8,0.9,1],

}
gsearch3 = GridSearchCV(estimator = XGBClassifier(**param), param_grid = param_test3, cv=10)

gsearch3.fit(X_train, y_train.values)


print("Meilleurs hyperparamètres :", gsearch3.best_params_)



Meilleurs hyperparamètres : {'gamma': 2, 'subsample': 1}


In [27]:
param = {
    "learning_rate" :0.3,
    "n_estimators":40,
    "max_depth":6,
    "gamma":2,
    "subsample":0.8,
    "colsample_bytree":1,
    "reg_lambda":0.6,
    "min_child_weight":2,
}

param_test4 = {
    "colsample_bytree":[0,1],
    "reg_lambda":[0.4,0.5,0.6],

}
gsearch4 = GridSearchCV(estimator = XGBClassifier(**param), param_grid = param_test4, cv=10)

gsearch4.fit(X_train, y_train.values)


print("Meilleurs hyperparamètres :", gsearch4.best_params_)


Meilleurs hyperparamètres : {'colsample_bytree': 1, 'reg_lambda': 0.5}


In [48]:
param = {
    "learning_rate" :0.3,
    "n_estimators":40,
    "max_depth":6,
    "gamma":2,
    "subsample":0.9,
    "colsample_bytree":0.8,
    "reg_alpha":0.7,
    "min_child_weight":2,
}

param_test5 = {
    "reg_alpha":[0.8,0.9,1],
}
gsearch5 = GridSearchCV(estimator = XGBClassifier(**param), param_grid = param_test5, cv=10)

gsearch5.fit(X_train, y_train.values)


print("Meilleurs hyperparamètres :", gsearch5.best_params_)


Meilleurs hyperparamètres : {'reg_alpha': 0.9}


In [61]:

param = {
    "learning_rate" :0.3,
    "n_estimators":40,
    "max_depth":6,
    "gamma":2,
    "subsample":0.9,
    "colsample_bytree":1,
    "reg_lambda":0.5,
    "reg_alpha":0.9,
    "min_child_weight":2,
    "nthread":3,
    "scale_pos_weight":1,
}

param_test6 = {
    "nthread":[2,3,4],
    "scale_pos_weight":[0,1,2],
}
gsearch6 = GridSearchCV(estimator = XGBClassifier(**param), param_grid = param_test6, cv=10)

gsearch6.fit(X_train, y_train.values)


print("Meilleurs hyperparamètres :", gsearch6.best_params_)



Meilleurs hyperparamètres : {'nthread': 2, 'scale_pos_weight': 1}


# Submission

In [175]:

xgb.fit(X_train, y_train)


y_proba = xgb.predict_proba(X_test)[:, 1]

threshold = 0.70
y_pred = (y_proba > threshold).astype(int)

In [176]:
submission = pd.read_csv("data/sample_submission.csv")
submission['TARGET'] = y_pred
submission.to_csv("submission_tard_test.csv", index=False)